In [85]:
%%time
import numpy as np
import pandas as pd
import os
os.chdir('E:\hc_gsm')
Gsm_Wb=pd.read_csv('查询GERAN外部小区.csv',encoding="gbk",usecols=['NAME','GERAN小区标识','位置区域码','GERAN频点','网络色码','基站色码','小区名称'],dtype={'GERAN小区标识':str,'位置区域码':str,'GERAN频点':str,'网络色码':str,'基站色码':str})
Gsm_Arfcn=pd.read_csv('查询GERAN_BCCH相邻频点.csv',encoding="gbk",usecols=['NAME','本地小区标识','BCCH组标识','GERAN频点'],dtype={'本地小区标识':str,'BCCH组标识':str,'GERAN频点':str})
Gsm_Info=pd.read_csv('GSM小区资源统计.csv',encoding="gbk",usecols=['BSC名称','基站索引','基站名称','小区索引','小区名称','本地小区标识','位置区码','小区CI','基站色码','网络色码','BCCH频点'],dtype={'位置区码':str,'小区CI':str,'基站色码':str,'网络色码':str,'BCCH频点':str})

Wall time: 5.19 s


In [86]:
Gsm_Wb['LAC_CI']=Gsm_Wb['位置区域码'].map(str)+"_"+Gsm_Wb['GERAN小区标识'].map(str)

In [87]:
Gsm_Info['LAC_CI']=Gsm_Info['位置区码'].map(str)+"_"+Gsm_Info['小区CI'].map(str)

In [88]:
Gsm_Wb1=pd.merge(Gsm_Wb,Gsm_Info,how='left',left_on='LAC_CI',right_on='LAC_CI')

In [89]:
Gsm_Wb1=Gsm_Wb1.dropna()

In [90]:
# 定义外部核查函数
def BCCH(a,b):
    if a==b:
        return 'BCCH正确'
    else:
        return 'BCCH错误'
def NCC(a,b):
    if a==b:
        return 'NCC正确'
    else:
        return 'NCC错误'
def BCC(a,b):
    if a==b:
        return 'BCC正确'
    else:
        return 'BCC错误'
Gsm_Wb1['频点是否正常']=Gsm_Wb1.apply(lambda x : BCCH(x['GERAN频点'],x['BCCH频点']),axis=1)
Gsm_Wb1['NCC是否正常']=Gsm_Wb1.apply(lambda x : NCC(x['网络色码_x'],x['网络色码_y']),axis=1)
Gsm_Wb1['BCC是否正常']=Gsm_Wb1.apply(lambda x : BCC(x['基站色码_x'],x['基站色码_y']),axis=1)
Gsm_Wb1['外部核查情况']=Gsm_Wb1['频点是否正常']+Gsm_Wb1['NCC是否正常']+Gsm_Wb1['BCC是否正常']

Gsm_Wb2=Gsm_Wb1[Gsm_Wb1['外部核查情况'].str.contains('错误')]
# 划分GSM外部错误类型
def mml(a,b,c,d,e,f,g):
    if g == 'BCCH错误NCC错误BCC错误':
        return f"""MOD GERANEXTERNALCELL:MCC="460",MNC="00",GERANCELLID={b},LAC={c},GERANARFCN={d},NETWORKCOLOURCODE={e},BASESTATIONCOLOURCODE={f};{{{a}}}"""
    elif g== 'BCCH错误NCC错误BCC正确':
        return f"""MOD GERANEXTERNALCELL:MCC="460",MNC="00",GERANCELLID={b},LAC={c},GERANARFCN={d},NETWORKCOLOURCODE={e};{{{a}}}"""
    elif g == 'BCCH错误NCC正确BCC错误':
        return f"""MOD GERANEXTERNALCELL:MCC="460",MNC="00",GERANCELLID={b},LAC={c},GERANARFCN={d},BASESTATIONCOLOURCODE={f};{{{a}}}"""
    elif g == 'BCCH错误NCC正确BCC正确':
        return f"""MOD GERANEXTERNALCELL:MCC="460",MNC="00",GERANCELLID={b},LAC={c},GERANARFCN={d};{{{a}}}"""
    elif g == 'BCCH正确NCC错误BCC错误':
        return f"""MOD GERANEXTERNALCELL:MCC="460",MNC="00",GERANCELLID={b},LAC={c},NETWORKCOLOURCODE={e},BASESTATIONCOLOURCODE={f};{{{a}}}"""
    elif g == 'BCCH正确NCC错误BCC正确':
        return f"""MOD GERANEXTERNALCELL:MCC="460",MNC="00",GERANCELLID={b},LAC={c},NETWORKCOLOURCODE={e};{{{a}}}"""
    else:
        return f"""MOD GERANEXTERNALCELL:MCC="460",MNC="00",GERANCELLID={b},LAC={c},BASESTATIONCOLOURCODE={f};{{{a}}}"""
    
Gsm_Wb2['MML']=Gsm_Wb2.apply(lambda x : mml(x['NAME'],x['小区CI'],x['位置区码'],x['BCCH频点'],x['网络色码_y'],x['基站色码_y'],x['外部核查情况']),axis=1)
Gsm_Wb2.to_excel('Gsm_Wb_外部核查结果.xlsx',index=None)


c:\program files (x86)\python37-32\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [91]:

Gsm_Wb2_Arfcn=Gsm_Wb1[Gsm_Wb1['频点是否正常'].str.contains('错误')]

In [92]:
# 筛选出包含GSM错误频点的LTE站点
Gsm_Arfcn=pd.merge(Gsm_Arfcn,Gsm_Wb2_Arfcn,how='left',on='NAME')

In [93]:
Gsm_Arfcn=Gsm_Arfcn.dropna(subset=['频点是否正常'])

In [94]:
# 定义删除所有GSM频点的函数
def mml1(h,i,j,k):
    return f"""RMV GERANNFREQGROUPARFCN:LOCALCELLID={h},BCCHGROUPID={i},GERANARFCN={j};{{{k}}}"""
# 添加正确的GSM频点
def mml2(h,i,j,k):
    return f"""ADD GERANNFREQGROUPARFCN:LOCALCELLID={h},BCCHGROUPID={i},GERANARFCN={j};{{{k}}}"""
# 删除冗余的GSM频点
def mml3(h,i,j,k):
    return f"""RMV GERANNFREQGROUPARFCN:LOCALCELLID={h},BCCHGROUPID={i},GERANARFCN={j};{{{k}}}"""
Gsm_Arfcn['mml1']=Gsm_Arfcn.apply(lambda y : mml1(y['本地小区标识_x'],y['BCCH组标识'],y['GERAN频点_x'],y['NAME']),axis=1)
Gsm_Arfcn['mml2']=Gsm_Arfcn.apply(lambda z : mml2(z['本地小区标识_x'],z['BCCH组标识'],z['BCCH频点'],z['NAME']),axis=1)
Gsm_Arfcn['mml3']=Gsm_Arfcn.apply(lambda x1 : mml3(x1['本地小区标识_x'],x1['BCCH组标识'],x1['GERAN频点_y'],x1['NAME']),axis=1)
Gsm_Arfcn.to_excel('Gsm_Arfcn.xlsx',index=None)

In [95]:
temp0=Gsm_Arfcn[['mml1']]
temp1=Gsm_Arfcn[['mml2']]
temp2=Gsm_Arfcn[['mml3']]

In [96]:
temp0=temp0.drop_duplicates()
temp1=temp1.drop_duplicates()
temp2=temp2.drop_duplicates()


In [97]:
temp0.rename(columns={'mml1':'MML'},inplace=True)
temp1.rename(columns={'mml2':'MML'},inplace=True)
temp2.rename(columns={'mml3':'MML'},inplace=True)

In [98]:
result=pd.concat([temp0,temp1,Gsm_Wb2[['MML']],temp2])
result.to_csv('4-2外部修改.txt',encoding='gbk',index=None,header=False,sep='?',quoting=3,doublequote=False,escapechar='?')